In [42]:
import torch
from models.FinetuneVTmodels import MIL_VT_FineTune
from models.MIL_VT import MIL_VT_small_patch16_384

# Create an instance of custom model
model = MIL_VT_FineTune()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()


Timm here
Missing Key not in pretrain model:  MIL_Prep.0.weight
Missing Key not in pretrain model:  MIL_Prep.0.bias
Missing Key not in pretrain model:  MIL_Prep.1.weight
Missing Key not in pretrain model:  MIL_Prep.1.bias
Missing Key not in pretrain model:  MIL_attention.0.weight
Missing Key not in pretrain model:  MIL_attention.0.bias
Missing Key not in pretrain model:  MIL_attention.1.weight
Missing Key not in pretrain model:  MIL_attention.1.bias
Missing Key not in pretrain model:  MIL_attention.4.weight
Missing Key not in pretrain model:  MIL_attention.4.bias
Missing Key not in pretrain model:  MIL_classifier.0.weight
Missing Key not in pretrain model:  MIL_classifier.0.bias
Removing key head.weight from pretrained checkpoint
Removing key head.bias from pretrained checkpoint
Removing key head_dist.weight from pretrained checkpoint
Removing key head_dist.bias from pretrained checkpoint
pos_embed:  384 576 1


MILVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0, inplace=False)
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop2): Dropout(p=0, inplace=False)
      )
      (ls2): Ide

In [59]:
from PIL import Image
from torchvision import transforms

image_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/image_set5/4_l4.jpeg'
image = Image.open(image_path)

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# Apply the transformation
input_image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

target_img_path = '/home/montasir/Desktop/Montasir/Spring 2024/Research and Experiments/Medical Imaging/MIL-VT-main/images/image_set5/0_l0.jpeg'
image2 = Image.open(target_img_path)

target_image = transform(image2).unsqueeze(0).to(device)

with torch.no_grad():
    target_embedding,_= model.forward_features(target_image)


In [36]:
target_embedding.shape

torch.Size([1, 384])

In [38]:
#Standard Gradient Descent

import torch
import torch.nn.functional as F

def optimize_embeddings(model, cur_input, target_emb, learning_rate, l2_dist_threshold, cosine_sim_threshold):
    """
    Adjusts initial input to match target embedding, stopping when thresholds are met.
    Args:
        model (torch.nn.Module): Model for generating embeddings.
        cur_input (torch.Tensor): Input tensor to be optimized.
        target_emb (torch.Tensor): Target embedding to match.
        learning_rate (float): Learning rate for gradient descent.
        l2_dist_threshold (float): Threshold for squared L2 distance.
        cosine_sim_threshold (float): Threshold for cosine similarity.
    Returns:
        torch.Tensor: Optimized input tensor.
        list: L1 distances over iterations.
        list: Cosine similarities over iterations.
        list: Losses over iterations.
    """
    org_input = cur_input.clone()
    max_val = max(cur_input.flatten()).item()
    min_val = min(cur_input.flatten()).item()

    squared_l2_distance = float('inf')
    cosine_sim_arr = []
    loss_arr = []
    l1_dist_arr = []

    iteration_count = 0

    while squared_l2_distance >= l2_dist_threshold or cosine_sim <= cosine_sim_threshold:
        cur_input = cur_input.clone().detach().requires_grad_(True)

        cur_emb,_ = model.forward_features(cur_input)

        loss = F.mse_loss(target_emb, cur_emb)
        loss_arr.append(loss.item())

        cur_input.grad = None
        loss.backward(retain_graph=True)
        grad = cur_input.grad

        updated_input = cur_input - learning_rate * grad
        updated_input = torch.clamp(updated_input, min_val, max_val)

        with torch.no_grad():
            updated_emb,_ = model.forward_features(updated_input)

        squared_l2_distance = torch.sum((target_emb - updated_emb)**2).item()

        updated_l1_dist = torch.sum(torch.abs(updated_input.detach() - org_input)).item()
        cosine_sim = F.cosine_similarity(target_emb, updated_emb)

        print(iteration_count, '\n')
        print("Squared L2 Distance:", squared_l2_distance)
        print("Cosine Similarity:", cosine_sim.detach().cpu().item())

        l1_dist_arr.append(updated_l1_dist)
        cosine_sim_arr.append(cosine_sim.detach().cpu().item())

        cur_input = updated_input
        iteration_count += 1

    return cur_input.detach(), l1_dist_arr, cosine_sim_arr, loss_arr


In [50]:
#PGD

import torch
import torch.nn.functional as F

def optimize_embeddings_pgd(model, cur_input, target_emb, learning_rate, epsilon, l2_dist_threshold, cosine_sim_threshold):
    """
    Adjusts initial input to match target embedding using Projected Gradient Descent.
    Args:
        model (torch.nn.Module): Model for generating embeddings.
        cur_input (torch.Tensor): Input tensor to be optimized.
        target_emb (torch.Tensor): Target embedding to match.
        learning_rate (float): Learning rate for gradient descent.
        epsilon (float): Epsilon value for controlling perturbations in PGD.
        l2_dist_threshold (float): Threshold for squared L2 distance.
        cosine_sim_threshold (float): Threshold for cosine similarity.
    Returns:
        torch.Tensor: Optimized input tensor.
        list: L1 distances over iterations.
        list: Cosine similarities over iterations.
        list: Losses over iterations.
    """
    org_input = cur_input.clone()

    squared_l2_distance = float('inf')
    cosine_sim_arr = []
    loss_arr = []
    l1_dist_arr = []

    iteration_count = 0
    while squared_l2_distance >= l2_dist_threshold or cosine_sim <= cosine_sim_threshold:
        cur_input = cur_input.clone().detach().requires_grad_(True)

        cur_emb,_ = model.forward_features(cur_input)

        loss = F.mse_loss(target_emb, cur_emb)
        loss_arr.append(loss.item())

        cur_input.grad = None
        loss.backward(retain_graph=True)
        grad = cur_input.grad

        updated_input = cur_input - learning_rate * grad

        # Perform projection to ensure the updated_input is within the feasible set
        projected_input = cur_input + torch.clamp(updated_input - cur_input, -epsilon, epsilon)

        with torch.no_grad():
            updated_emb,_ = model.forward_features(projected_input)

        squared_l2_distance = torch.sum((target_emb - updated_emb)**2).item()

        updated_l1_dist = torch.sum(torch.abs(projected_input.detach() - org_input)).item()
        cosine_sim = F.cosine_similarity(target_emb, updated_emb)

        print(iteration_count, '\n')
        print("Squared L2 Distance:", squared_l2_distance)
        print("Cosine Similarity:", cosine_sim.detach().cpu().item())

        l1_dist_arr.append(updated_l1_dist)
        cosine_sim_arr.append(cosine_sim.detach().cpu().item())

        cur_input = projected_input
        iteration_count += 1

    return cur_input.detach(), l1_dist_arr, cosine_sim_arr, loss_arr


In [53]:
def inverse_normalize():
    """
    Returns an inverse normalization transformation.
    Returns:
        transforms.Normalize: Transformation to apply inverse normalization.
    """
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    return transforms.Normalize(mean=[-m/s for m, s in zip(mean, std)], std=[1/s for s in std])


In [60]:
from torchvision.utils import save_image
#optimized_image,_,_,_, = optimize_embeddings(model, input_image, target_embedding, learning_rate=0.09, l2_dist_threshold=80, cosine_sim_threshold=.85)
optimized_image,_,_,_, = optimize_embeddings_pgd(model, input_image, target_embedding, learning_rate=0.03,epsilon= 0.03,l2_dist_threshold=80, cosine_sim_threshold=.85)
optimized_image_inv = inverse_normalize()(optimized_image)
save_image(optimized_image_inv, './output/PGD/l4>l0.png')




0 

Squared L2 Distance: 459.6717529296875
Cosine Similarity: 0.5673993825912476
1 

Squared L2 Distance: 420.8707580566406
Cosine Similarity: 0.5945403575897217
2 

Squared L2 Distance: 390.40130615234375
Cosine Similarity: 0.6167594790458679
3 

Squared L2 Distance: 365.8084411621094
Cosine Similarity: 0.6351531744003296
4 

Squared L2 Distance: 347.01025390625
Cosine Similarity: 0.6496411561965942
5 

Squared L2 Distance: 333.3671569824219
Cosine Similarity: 0.6605674028396606
6 

Squared L2 Distance: 323.32305908203125
Cosine Similarity: 0.668937623500824
7 

Squared L2 Distance: 315.42987060546875
Cosine Similarity: 0.6757266521453857
8 

Squared L2 Distance: 308.7862548828125
Cosine Similarity: 0.6815564632415771
9 

Squared L2 Distance: 302.9274597167969
Cosine Similarity: 0.6867586970329285
10 

Squared L2 Distance: 297.6171569824219
Cosine Similarity: 0.6915097236633301
11 

Squared L2 Distance: 292.7238464355469
Cosine Similarity: 0.6959133148193359
12 

Squared L2 Distance: 

In [1]:
import open_clip
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)
#model, preprocess = clip.load("ViT-L/14", device=device)
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', device=device)

/home/montasir/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model